### Why memory is important?
    - chat models totally rely on memory
    - memory is the key to understand the context
    
### How to handle memory on LLMs?
    1. put it in the prompt
    2. have external lookup

### What is the best way to handle memory?
    - put it in the prompt is easier but is not the best way, because as conversation goes on, the prompt will be too long
      but we can recosntruct the memory from the conversation in different ways to optimize the memory as,
            1. buffer the conversation
            2. summarize the conversation
            3. buffer the conversation with window  
            4. buffer the conversation with window and summarization
            5. use knowledge graph 

In [27]:
import tiktoken
from langchain.chains import ConversationChain
import yaml, os, tiktoken, langchain, textwrap
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferMemory, \
                                                 ConversationSummaryMemory, \
                                                 ConversationBufferWindowMemory, \
                                                 ConversationSummaryBufferMemory, \
                                                 ConversationKGMemory

In [32]:
!pip install tiktoken

In [2]:
with open('cadentials.yaml') as f:
    cadentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = cadentials['OPENAI_API_KEY']
enc = tiktoken.get_encoding("cl100k_base")

In [3]:
client = ChatOpenAI(
                    openai_api_key = os.environ['OPENAI_API_KEY'],
                    model = 'gpt-3.5-turbo',
                    temperature=0,
                    max_tokens=500
                    )

## 1. ConversationBufferMemory

In [21]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
                                llm=client,
                                memory=memory,
                                verbose=True
                                )

In [22]:
template = conversation.prompt.template
print(template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [23]:
buffer = conversation.memory.buffer
print(buffer)

In [24]:
query = "Do you know about machine learning ?"

In [25]:
conversation.run(query)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Do you know about machine learning ?
AI:

> Finished chain.


'Yes, I am familiar with machine learning. It is a subfield of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. Machine learning involves training a model on a large dataset and using statistical techniques to enable the model to learn patterns and relationships within the data. This allows the model to make accurate predictions or decisions on new, unseen data. Machine learning has applications in various fields such as image recognition, natural language processing, and recommendation systems.'

In [12]:
buffer = conversation.memory.buffer
print(buffer)

Human: Hi, I'm Isuru !
AI: Hello Isuru! How can I assist you today?
Human: Do you know about machine learning ?
AI: Yes, I am familiar with machine learning. It is a field of study that focuses on developing algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. Machine learning algorithms can analyze and interpret large amounts of data to identify patterns and make accurate predictions or decisions. It is widely used in various applications such as image recognition, natural language processing, and recommendation systems.


#### write the chat in a better way


In [39]:
memory = ConversationBufferMemory()

conversation = ConversationChain(
                                llm=client,
                                memory=memory,
                                verbose=False
                                )

In [40]:
query = input("Enter your query : ")
query

'f'

In [41]:
iteration = 1

while True:
    query = input("Enter your query : ")
    if query == 'exit':
        break 

    else:
        template = conversation.prompt.template
        buffer = conversation.memory.buffer
        print("####################################################################################################") 
        print("########################################## ITERATION : {} ##########################################".format(iteration))
        print(template.format(history=buffer, input=query))
        print('\n')
        tokens = len(enc.encode(template.format(history=buffer, input=query)))
        print(f"TOKENS : {tokens}")
        print('\n')
        response = conversation.predict(input=query)
        print(f"RESPONSE : {response}")
        print('\n')

        iteration += 1

####################################################################################################
########################################## ITERATION : 1 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hello, I am zuu 
AI:


TOKENS : 63


RESPONSE : Hello Zuu! How can I assist you today?


####################################################################################################
########################################## ITERATION : 2 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current convers

#### you can see it's stacking the conversation in the prompt

In [42]:
template = conversation.prompt.template
buffer = conversation.memory.buffer
print(template.format(history=buffer, input=query))

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hello, I am zuu 
AI: Hello Zuu! How can I assist you today?
Human: do you know about machine klearning ?
AI: Yes, I am familiar with machine learning. It is a field of study that focuses on developing algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. Machine learning involves training a model on a dataset and using that trained model to make predictions or decisions on new, unseen data. It has applications in various domains such as image recognition, natural language processing, and recommendation systems. Is there anything specific you would like to know about machine learning?
Human: how it diferes from deep learning ?
AI: Machine learning and deep learni

## 2. ConversationSummaryMemory

In [45]:
sumary_memory = ConversationSummaryMemory(llm=client)
conversation = ConversationChain(
                                llm=client,
                                memory=sumary_memory,
                                verbose=False
                                )

In [46]:
iteration = 1

while True:
    query = input("Enter your query : ")
    if query == 'exit':
        break 

    else:
        template = conversation.prompt.template
        buffer = conversation.memory.buffer
        print("####################################################################################################") 
        print("########################################## ITERATION : {} ##########################################".format(iteration))
        print(template.format(history=buffer, input=query))
        print('\n')
        tokens = len(enc.encode(template.format(history=buffer, input=query)))
        print(f"TOKENS : {tokens}")
        print('\n')
        response = conversation.predict(input=query)
        print(f"RESPONSE : {response}")
        print('\n')

        iteration += 1

####################################################################################################
########################################## ITERATION : 1 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I am isuru !
AI:


TOKENS : 63


RESPONSE : Hello Isuru! How can I assist you today?


####################################################################################################
########################################## ITERATION : 2 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conve

## 3. ConversationBufferWindowMemory

In [47]:
window_memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
                                llm=client,
                                memory=window_memory,
                                verbose=False
                                )

In [48]:
template = conversation.prompt.template
buffer = conversation.memory.buffer
print(template.format(history=buffer, input=query))

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: exit
AI:


In [49]:
iteration = 1

while True:
    query = input("Enter your query : ")
    if query == 'exit':
        break 

    else:
        template = conversation.prompt.template
        buffer = conversation.memory.buffer
        print("####################################################################################################") 
        print("########################################## ITERATION : {} ##########################################".format(iteration))
        print(template.format(history=buffer, input=query))
        print('\n')
        tokens = len(enc.encode(template.format(history=buffer, input=query)))
        print(f"TOKENS : {tokens}")
        print('\n')
        response = conversation.predict(input=query)
        print(f"RESPONSE : {response}")
        print('\n')

        iteration += 1

####################################################################################################
########################################## ITERATION : 1 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi
AI:


TOKENS : 58


RESPONSE : Hello! How can I assist you today?


####################################################################################################
########################################## ITERATION : 2 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi
A

## 4. ConversationSummaryBufferMemory

In [18]:
summary_buffer_memory = ConversationSummaryBufferMemory(
                                                        llm=client,
                                                        max_token_limit=50        
                                                        )
conversation = ConversationChain(
                                llm=client, 
                                verbose=False, 
                                memory=summary_buffer_memory
                                )

In [19]:
iteration = 1

while True:
    query = input("Enter your query : ")
    if query == 'exit':
        break 

    else:
        template = conversation.prompt.template
        buffer = conversation.memory.moving_summary_buffer
        print("####################################################################################################") 
        print("########################################## ITERATION : {} ##########################################".format(iteration))
        print(template.format(history=buffer, input=query))
        print('\n')
        tokens = len(enc.encode(template.format(history=buffer, input=query)))
        print(f"TOKENS : {tokens}")
        print('\n')
        response = conversation.predict(input=query)
        print(f"RESPONSE : {response}")
        print('\n')

        iteration += 1

####################################################################################################
########################################## ITERATION : 1 ##########################################
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: do you know about machine learning ?
AI:


TOKENS : 63


RESPONSE : Yes, I am familiar with machine learning. It is a subfield of artificial intelligence that focuses on the development of algorithms and models that allow computers to learn and make predictions or decisions without being explicitly programmed. Machine learning involves training a model on a large dataset and using statistical techniques to enable the model to learn patterns and relationships within the data. This allows the model to make accurate predictions or dec

## 5. Conversation Knowledge Graph Memory

In [29]:
with open('/Users/1zuu/Desktop/LLM RESEARCH/LLMPro/cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['AD_OPENAI_API_KEY'] = credentials['AD_OPENAI_API_KEY']

In [41]:
client = AzureChatOpenAI(
                    deployment_name=credentials['AD_DEPLOYMENT_ID'],
                    model=credentials['AD_ENGINE'],
                    api_key=credentials['AD_OPENAI_API_KEY'],
                    api_version=credentials['AD_OPENAI_API_VERSION'],
                    azure_endpoint=credentials['AD_OPENAI_API_BASE']
                    )

In [43]:
template = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:

{history}

Conversation:
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=template
)

conversation_with_kg = ConversationChain(
                                        llm=client, 
                                        verbose=True, 
                                        prompt=prompt,
                                        memory=ConversationKGMemory(llm=client)
                                    )

In [44]:
iteration = 1

while True:
    query = input("Enter your query : ")
    if query == 'exit':
        break 

    else:
        # template = conversation.prompt.template
        # buffer = conversation.memory.moving_summary_buffer
        print("####################################################################################################") 
        print("########################################## ITERATION : {} ##########################################".format(iteration))
        # print(template.format(history=buffer, input=query))
        # print('\n')
        # tokens = len(enc.encode(template.format(history=buffer, input=query)))
        # print(f"TOKENS : {tokens}")
        print('\n')
        response = conversation_with_kg.predict(input=query)
        print(f"RESPONSE : {response}")
        print('\n')

        iteration += 1

####################################################################################################
########################################## ITERATION : 1 ##########################################




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know. The AI ONLY uses information contained in the "Relevant Information" section and does not hallucinate.

Relevant Information:



Conversation:
Human: hi
AI:

> Finished chain.
RESPONSE : Hello! It's nice to meet you. How can I assist you today?


####################################################################################################
########################################## ITERATION : 2 ##########################################




> Entering new ConversationChain chain...
Pr

In [45]:
conversation_with_kg.memory.kg.get_triples()

[('Isuru', 'a person', 'is'),
 ('Isuru', 'machine learning', 'likes'),
 ('Machine learning', 'deep learning', 'is a subset of'),
 ('Machine learning', 'a wider range of applications', 'can be used for'),
 ('Machine learning',
  'a broader range of techniques and algorithms',
  'encompasses'),
 ('Deep learning', 'neural networks with multiple layers', 'uses'),
 ('Deep learning',
  'image recognition and natural language processing',
  'is used for')]